# Basics of Computational Fluid Dynamics

We have all the tools to address a physical problem of relevance in all the sciences and engineering: how does a fluid behave?

From the perspective we are using in this course we want to know what is the equation that should encode a fluid's behavior. That equation is the Navier-Stokes equation.

$$
\frac{\partial\vec{V}}{\partial t}+ (\vec{V}\cdot\nabla)\vec{V} = -\frac{1}{\rho}\nabla p + \nu\nabla^2\vec{V}
$$

where $\vec{V}$ is the velocity field describing the fluid motion, $\rho$ is the density, $p$ is the pressure and $\nu$ is a constant known as viscosity.

Just to make the discussion a bit more concrete I will write this equation in 1D

$$
\frac{\partial V_x}{\partial t} + V_x\frac{\partial V_x}{\partial x} = -\frac{1}{\rho}\frac{\partial p}{\partial x} + \nu\frac{\partial^2 V_x}{\partial x^2}
$$

In this equation $V_x\frac{\partial V_x}{\partial x}$ represents convection and $\nu\frac{\partial^2 V_x}{\partial x^2}$ represents difussion.

## The plan

The plan to solve this problem is write each derivative as finite differences expressions. This will allows us to discretize
time and space (i.e. define $\Delta t$ and $\Delta x$) to find how our function $V_x$ evolves in time and space


We are going to study the following processes in increasing complexity

- Linear convection
- Non-linear convection
- Difussion
- Non-linear convection + difussion (Burgers equation)

Please note tat this notebook follows the structure of Prof. Lorena Barba's 12 steps to Navier Stokes:
http://nbviewer.ipython.org/github/barbagroup/CFDPython/tree/master/lessons/


Diferencias finitas: 
*DISCRETIZAR r,t
*CONDICIONES INICIALES/ C.FRONTERA
*EVOLUCIONAR EN EL TIEMPO

### Linear convection

We want to solve the following equation

$$
\frac{\partial u}{\partial t} + c\frac{\partial u}{\partial x} = 0
$$

How do we do it? We discretize the derivatives:

$$
\frac{\partial u}{\partial x} \approx \frac{u_{i}^j -  u_{i-1}^j}{\Delta x}
$$

$$
\frac{\partial u}{\partial t} \approx \frac{u_i^{j+1} -  u_i^j}{\Delta t}
$$

superinidice denota tiempo, subindice denota posicion

So we can rewrite the equation
$$
\frac{u_i^{j+1} -  u_i^j}{\Delta t} = - c \frac{u_{i}^j -  u_{i-1}^j}{\Delta x}
$$

$$
u_{i}^{j+1} = u_{i}^{j} - c\frac{\Delta t}{\Delta x}(u_i^j - u_{i-1}^j) 
$$

We now go for the implementation in Python.

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [19]:
n_x = 80 #discretizar en x
n_t = 300 #discretiza en t, no necesariamente son iguales los terminos

x = linspace(0, 2.0, n_x)
u = ones(n_x) #inicializa a u con un arreglo de 1, no 0 

dx = x[1]-x[0] #el paso de la malla, puede ser irregular po eso es una funcion 
dt = 0.001
c = 1.0

#but now the initial condition is not flat
u[where((x<1.25) & (x>0.75))] = 2.0 #funcion escalon, el where saca el indice donde dicha condicion es verdadera

In [21]:
for n in range(n_t):  # loop over time, evolucion del tiempo y el espacio es un for anidado 
    u_past = u.copy() # el .copy() garantiza que u_past nuevo sea distinto a u, es igual que usar un list(u).  
    for i in range(1,n_x-1): #loop over space, poemos la ecuacion de discretizacion 
        u[i] = u_past[i] - c*dt/dx*(u_past[i]-u_past[i-1])

## Non-Linear convection

Now the equation is a bit different
$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = 0
$$

In [34]:
n_x = 80
n_t = 300

x = linspace(0, 2.0, n_x)
u = ones(n_x)

dx = x[1]-x[0]
dt = 0.001

print dx/dt

#but now the initial condition is not flat
u[where((x<1.25) & (x>0.75))] = 2.0

25.3164556962


In [44]:
for n in range(n_t):  # loop over time
    u_past = list(u)
    for i in range(1,n_x): #loop over space
        u[i] = u_past[i] - u_past[i]*dt/dx*(u_past[i]-u_past[i-1])

## Difusion

Now we want to solve the equation
$$
\frac{\partial u}{\partial t} = \nu\frac{\partial^2 u}{\partial x^2} 
$$

We already had the discretized version


$$
\frac{\partial u}{\partial t} \approx \frac{u_{i}^{j+1} - u_{i}^{j}}{\Delta t}
$$

$$
\frac{\partial^2 u}{\partial x^2} \approx \frac{u_{i+1}^j - 2 u_{i}^{j} + u_{i-1}^{j}}{(\Delta x)^2}
$$

And the finite differences scheme can be found
$$
\frac{u_{i}^{j+1} - u_{i}^{j}}{\Delta t} = \nu \frac{u_{i+1}^j - 2 u_{i}^{j} + u_{i-1}^{j}}{(\Delta x)^2} 
$$

$$
u_{i}^{j+1} = \nu\alpha u_{i+1}^{j} + (1 - 2\nu\alpha)u_{i}^{j} + \nu\alpha u_{i-1}^{j}
$$

where $\alpha = \Delta t/ (\Delta x)^2$.

Vamos a utilizar los mismos n_x y n_t del ejemplo en clase, discretizando para x y t. Asi mismo para c y sigma. 

In [46]:
n_x = 80
n_t = 100

c = 1.0
nu = 0.3
sigma = 0.2 #sigma is a parameter to ensure \alpha\nu < 0.5 garantiza que converge para que no se pierda un elemento de un orde

x = linspace(0, 2.0, n_x)
dx = x[1]-x[0]


dt = sigma*dx**2/nu #dt is defined using sigma, despejar de la ec de alpha
alpha = dt/dx**2
print dt

u = ones(n_x)

u[where((x<1.25) & (x>0.75))] = 2.0

0.000427281952679


In [58]:
for n in range(n_t):  # loop over time
    u_past = u.copy() 
    for i in range(1,n_x-1): #loop over space
        u[i] = nu * alpha * u_past[i+1]  + (1.0 - 2.0*nu*alpha)*u_past[i] + nu*alpha*u_past[i-1] #copiar la funcion

In [1]:
%pylab inline
import numpy as np


Populating the interactive namespace from numpy and matplotlib


### Ecuacion Difusion en Reactor Tubular

Tenemos las ecuacione que describen el intercambio de calor entre 2 fluidos en un reactor tubular, considerando despreciable el ancho del mismo, de la siguiente manera:
Es fácil ver que ya que cr=0 para r=0, entonces crr cuando r tiene a 0 es un límite de la forma 0/0. De ello, podemos usar la regla de l'opital derivando arriba y abajo de lo cual obtenemos rápidamente que podemos reemplazar cr/r=crr para r tendiendo a 0.

Por un proceso similar es también fácil ver que ya que Tr=0 en r=0 entonces Trr=Trr.

De esto las nuevas funciones para r=0 van a ser:
ct=−vcz+D(czz+2crr)−r(c,T)
ct=−vTz+λpcpD(Tzz+2Trr)−ΔHpcpr(c,T)

In [2]:
#definicion de constantes, variables y pasos de cada ecuación diferencial
n_x = 100
n_t = 100
L = 30 #esos son los valores que toca ver para un caso determinado
R = 1 # radio reservorio

r = np.linspace(0.0,R,n_x)
z = np.linspace(0.0,L,n_x)

dr = r[1]-r[0]
dz = z[1]-z[0]

#trate de hacer el dt como en la wave equation pero no veo como despejarlo, entonces lo definiremos a ojo
dt = 0.01
d_H = 1 #entalpia
C_p = 1 #calor especifico
rho = 1 #densidad
T_w = 2 #temperatura del reservorio
lamda = 1
k_0 = 10
E = 10
D = 1 #difusividad



In [3]:
#Funcion r 
def rf(c1, T1): 
    return k_0 + exp(-E/(R*T1))*(c1**2)

#Funcion v
def v_over_vmax(r1):
    return (1-(r1/R)**2)

In [15]:
#declaramos ahora las discretizaciones, inicializamos los arreglos para las soluciones encontradas
T_0= np.ones((n_x, n_x))
c_0= np.ones((n_x, n_x))

T_past = np.zeros((n_x, n_x))
T_pres = T_in

c_past = np.zeros((n_x, n_x))
c_pres = c_in

#ahora definimos las condiciones de frontera en cada caso
                           
#f es la funcion que queremos derivar, i es r, j es z. Estas son las primeras derivadas

def derivada1_r(f,i,j,h,lamda):
                
    if(i==len(f)): #cuando R=r
        return (h/lamda)*(Tw - f(i,j))
    elif(i == 0):
        return 0
    else:
        return (f(i+1,j)-f(i-1, j))/(2*dr) 

def derivada1_z(f,i,j):
                
    if(j==len(f)):
        return 0
    elif(j == 0):
        return 0
    else:
        return (f(i,j+1)-f(i,j-1))/(2*dz) 

#f es la funcion que queremos derivar, i es r, j es z. Estas son las segundas derivadas. 
def derivada2_r(f, i, j):
    
    if(i==len(f)):
        return (h/lamda)*(Tw - f(i,j))
    elif(i == 0):
        return 0
    return (f(i+1,j)+ 2*f(i,j)-f(i-1,j))/(2*dr) 

def derivada2_z(f,i,j):
         
    if(j == len(f)):
        return 0
    elif(j == 0):
        return 0
    return (f(i,j+1)+2*f(i,j)-f(i,j-1))/(2*dz)  

#aca no estoy segura si hacerlas por separado o en una sola funcion que derive para cada caso

In [28]:
#aca es que tengo problemas, este for recorre en t y da los cambios para c y T. 
for t in range(n_t):
    T_past = T_pres.copy()
    c_past = c_pres.copy()
    for i in range(1,n_x-1):
        for j in range(1,n_x-1): #FALTAN CONDICIONES DE FRONTERA
            a = 0
            b = 0
            c = 0
            d = 0
            if(r[i] == 0):
                derivaa1_r(c_past,i,j) == a
                derivada1_r(T_past,i,j) == a
            elif(r[i] == R):
                derivada1_r(c_past,i,j) == b
                
            if(z[j] == L):
                c_pres == c
                T_pres == d
         
            else:
                c_pres = dt*(c_past[i,j]) - v_over_vmax(r[i])*derivada1_z(c_past,i,j)  + D*(derivada2_z(c_past,i,j) + derivada2_r(c_past,i,j) + derivada1_r(c_past,i,j)/r[i])- rf(c_past[i,j],T_past[i,j])                    
                T_pres = dt*(T_past[i,j]) - v_over_vmax(r[i])*derivada1_z(T_past,i,j)  + (lamda/rho*C_p)*(derivada2_z(T_past,i,j) + derivada2_r(T_past,i,j) + derivada1_r(T_past,i,j)/r[i])- (d_H/rho*C_p)* rf(c_past[i,j],T_past[i,j])
            
         

TypeError: 'numpy.ndarray' object is not callable

#Exercise

1. Write a program in C to compute the solution of the 1D Burgers Equation.
Make a program in Python to make a movie of the time evolution. Use the same initial conditions as above. 

2. Write a program in C to compute the solution of the 2D Burguers Equation.
Make a program in Python to make a movie of the time evolution. Follow this: http://nbviewer.ipython.org/github/barbagroup/CFDPython/blob/master/lessons/10_Step_8.ipynb